In [8]:
# Import required libraries and setup Snowflake connection
# Configure logging for the Agent Gateway
# Set up Snowflake connection parameters from environment variables
# These parameters are loaded from .env file or environment variables
# Create a Snowflake session using the configured parameters

from agent_gateway import Agent
from agent_gateway.tools import CortexSearchTool, CortexAnalystTool, PythonTool
from snowflake.snowpark import Session
import os
import logging

logging.getLogger("AgentGatewayLogger").setLevel(logging.INFO)

connection_parameters = {
    "account": os.getenv("SNOWFLAKE_ACCOUNT"),
    "user": os.getenv("SNOWFLAKE_USER"),
    "password": os.getenv("SNOWFLAKE_PASSWORD"),
    "role": os.getenv("SNOWFLAKE_ROLE"),
    "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE"),
    "database": os.getenv("SNOWFLAKE_DATABASE"),
    "schema": os.getenv("SNOWFLAKE_SCHEMA"),
}

session = Session.builder.configs(connection_parameters).create()

In [9]:
from IPython.display import HTML
import requests, json
import yfinance as yf
import pandas as pd

# Import required libraries for stock data retrieval and display
# StockTool class implementation for fetching real-time stock market data

class StockTool:
    def __init__(self) -> None:
        pass

    def stock_search(self, ticker: str) -> str:
        """Get current stock information for a given ticker"""
        try:
            # Get stock info using fast_info
            stock = yf.Ticker(ticker)
            info = stock.fast_info

            # Create Yahoo Finance URL for the ticker
            url = f"https://finance.yahoo.com/quote/{ticker}"
            # Create HTML link
            html_link = f'<a href="{url}" target="_blank">{url}</a>'

            # Extract relevant information
            current_price = info.last_price
            previous_close = info.previous_close
            market_cap = info.market_cap

            # Format market cap to be more readable
            if isinstance(market_cap, (int, float)):
                market_cap = f"${market_cap/1000000000:.2f}B"

            # Create detailed response
            response = (
                f"Stock information for {ticker}:\n"
                f"Current Price: ${current_price}\n"
                f"Previous Close: ${previous_close}\n"
                f"Market Cap: {market_cap}\n"
                f"Exchange Timezone: {info.timezone}\n"
                f"\nSource: {html_link}"
            )

            # Display HTML link in notebook
            display(HTML(html_link))

            return response
        except Exception as e:
            return f"Error fetching data for {ticker}: {str(e)}"


# Stock tool configuration
stock_config = {
    "tool_description": "searches for current stock market data based on ticker symbol",
    "output_description": "current stock price, previous close, and market cap information",
    "python_func": StockTool().stock_search,
}

In [10]:
# Configuration for Cortex Analyst Tool
# Used for analyzing sales metrics and data from Snowflake
analyst_config = {
    "semantic_model": "SALES_MODEL.yaml",
    "stage": "ANALYST",
    "service_topic": (
        "Comprehensive Sales Analytics and Marketing Performance: "
        "analyzing orders, revenue, marketing spend, customer segments, and promotions "
        "across various geographies and product categories."
    ),
    "data_description": (
        "This dataset (SALES_DATA) provides a unified view of customer orders—"
        "including demographics, revenue, marketing spend (ad/promotion costs), and "
        "payment details. With dimensional coverage for geography (COUNTRY), promotions (SALES_PROMOTION), "
        "product categories (PRODUCT_CATEGORY), and customer segments (CUSTOMER_SEGMENT), "
        "it facilitates queries related to total orders, revenue, promotional effectiveness, "
        "marketing ROI, and product lifecycles. Time-based analysis is enabled via ORDER_DATE, "
        "and measures such as REVENUE, MARKETING_SPEND, and PRODUCT_AGE allow for "
        "in-depth exploration of sales performance."
    ),
    "snowflake_connection": session,
}


# Configuration for Knowledge Base Article (KBA) Search Tool
# Used for searching and retrieving information from knowledge base articles
search_config = {
    "service_name": "KBA",  # matches existing service name
    "service_topic": (
        "Comprehensive Knowledge Base on Product Campaigns, Launches, and Loyalty Programs "
        "across various markets and categories."
    ),
    "data_description": (
        "A suite of internal documentation covering upcoming marketing campaigns, product launches, "
        "payment options, loyalty initiatives, and regional strategies. These articles provide "
        "implementation details, timelines, eligibility requirements, and promotional insights "
        "for diverse categories such as Books, Beauty, and Toys."
    ),
    "retrieval_columns": ["CHUNK"],
    "snowflake_connection": session,
}


# Configuration for Stock Market Data Tool
# Used for real-time stock market data retrieval
stock_config = {
    "tool_description": "searches for current stock market data based on ticker symbol",
    "output_description": "current stock price, previous close, and market cap information",
    "python_func": StockTool().stock_search,
}

In [ ]:
# Initialize individual tools with their respective configurations
# Create an agent that combines all tools for unified access
# Set max_retries to 3 for handling temporary failures

sales = CortexAnalystTool(**analyst_config)
kba = CortexSearchTool(**search_config)
stock_search = PythonTool(**stock_config)

snowflake_tools = [sales, kba, stock_search]
agent = Agent(snowflake_connection=session, tools=snowflake_tools, max_retries=3)

## Structured Data

In [12]:
agent("what is total revenue for the toy category?")

INFO:AgentGatewayLogger:running SALES_MODEL_cortexanalyst task


'The total revenue for the toy category is $428,441.00.'

In [13]:
agent("break it out by country")


INFO:AgentGatewayLogger:running SALES_MODEL_cortexanalyst task


'The total revenue for the toy category broken out by country is as follows: UK - $75,120, Australia - $63,762, France - $62,887, Germany - $62,206, USA - $58,041, Japan - $57,221, Canada - $49,204.'

## Unstructured Data

In [16]:
agent("Which country is the beauty holiday collection launching in")

INFO:AgentGatewayLogger:running SALES_MODEL_cortexanalyst task
ERROR:AgentGatewayLogger:Your request is unclear. Consider rephrasing your request to one of the following suggestions:['Which country had the highest revenue for beauty products last holiday season?', 'Which country had the most orders for beauty products last holiday season?', 'Which country had the highest marketing spend for beauty products last holiday season?']
ERROR:AgentGatewayLogger:{'content': [{'text': "I apologize, but the question 'Which country is the "
                      "beauty holiday collection launching in?' is unclear "
                      'because it does not specify any data points or criteria '
                      'that can be used to determine the launch country from '
                      'the given schema. The schema does not contain '
                      'information about product launches or specific '
                      'collections.',
              'type': 'text'},
             {'s

'Germany'

## Unstructured Data + Structured Data

In [17]:
agent("Which country is the beauty holiday collection launching in? What is the total revenue for that country?")

INFO:AgentGatewayLogger:running SALES_MODEL_cortexanalyst task
ERROR:AgentGatewayLogger:Unable to generate a valid SQL Query. I apologize, but the question 'Which country is the beauty holiday collection launching in?' is unclear because it does not specify any data points or criteria that can be used to determine the launch country from the given schema. The schema does not contain information about product launches or specific collections.
ERROR:AgentGatewayLogger:{'content': [{'text': "I apologize, but the question 'Which country is the "
                      "beauty holiday collection launching in?' is unclear "
                      'because it does not specify any data points or criteria '
                      'that can be used to determine the launch country from '
                      'the given schema. The schema does not contain '
                      'information about product launches or specific '
                      'collections.',
              'type': 'text'}],
 '

'The beauty holiday collection is launching in Germany. The total revenue for Germany is $392,196.'

## Stock Tool

In [18]:
agent("What is the stock price of Hasbro and what was the total revenue in the toy category?")

INFO:AgentGatewayLogger:running stock_search task
INFO:AgentGatewayLogger:running SALES_MODEL_cortexanalyst task


'The stock price of Hasbro is $59.56, and the total revenue in the toy category from January 1, 2023, to January 1, 2025, is $428,441.'

In [19]:
agent("What is the stock price of Nvidia?")



INFO:AgentGatewayLogger:running stock_search task


'The current stock price of Nvidia is $124.65.'